# Clustering

In [1]:
import gc
import pandas as pd
import glob
import sklearn
import numpy as np

In [2]:
dcFiles=glob.glob("../Dc/*")
diFiles=glob.glob("../Di/*")

In [3]:
trainIndexFiles=glob.glob("../Train/*")
testIndexFiles=glob.glob("../Test/*")

In [4]:
dcFiles

['../Dc/Jnk.csv',
 '../Dc/IRF4.csv',
 '../Dc/LEF1.csv',
 '../Dc/Ikk2.csv',
 '../Dc/CTNNB1.csv',
 '../Dc/Erk.csv',
 '../Dc/MYC.csv']

In [5]:
trainIndexFiles

['../Train/LEF1.index.csv',
 '../Train/MYC.index.csv',
 '../Train/IRF4.index.csv',
 '../Train/Jnk.index.csv',
 '../Train/Ikk2.index.csv',
 '../Train/Erk.index.csv',
 '../Train/CTNNB1.index.csv']

# Setting indexes to read

In [6]:
irf4Indexes=pd.read_csv('../Train/MYC.index.csv',sep=";")
irf4Indexes.head()

,0,1
0,102,1.0
1,103,1.0
2,104,1.0
3,105,1.0
4,106,1.0


In [7]:
type(irf4Indexes['0'])

pandas.core.series.Series

In [8]:
irf4IndesxList=list(irf4Indexes['0'])

In [9]:
len(irf4IndesxList)

5038

In [11]:
irf4Dc=pd.read_csv('../Dc/IRF4.csv',sep=";")

In [12]:
irf4Dc[irf4Dc.columns[5:]].loc[irf4IndesxList].head()

,X0min_neg_effect-X0min_neg_cause,X15min_effect-X0min_neg_cause,X15min_effect-X15min_cause,X30min_effect-X0min_neg_cause,X30min_effect-X15min_cause,X30min_effect-X30min_cause,X90min_effect-X0min_neg_cause,X90min_effect-X15min_cause,X90min_effect-X30min_cause,X90min_effect-X90min_cause,...,X240min_effect-X240min_cause,X360min_effect-X0min_neg_cause,X360min_effect-X15min_cause,X360min_effect-X30min_cause,X360min_effect-X90min_cause,X360min_effect-X120min_cause,X360min_effect-X180min_cause,X360min_effect-X210min_cause,X360min_effect-X240min_cause,X360min_effect-X360min_cause
102,0.586861,0.229680,0.699956,0.442469,0.234291,0.480270,-0.113705,0.021928,-0.093741,0.429678,...,2.388120,-0.245855,-0.065930,-0.264162,0.177189,0.772294,0.640198,0.274996,-0.348037,1.969472
103,0.799353,0.761610,1.062872,-0.149231,-0.140814,0.268250,0.109091,0.121278,-0.086808,0.148070,...,0.380632,0.328329,0.292574,-0.103212,0.051283,0.316917,0.272946,0.198828,0.115947,0.692113
104,0.753384,-0.349794,1.318468,0.211955,-0.134256,0.461882,0.439737,-0.665943,0.172277,1.037117,...,1.134065,0.288913,-1.337756,-0.014677,1.004365,1.180434,0.860087,-0.003583,-0.943614,5.211742
105,0.959816,-0.160661,0.771176,0.170137,-0.244917,0.490405,-0.039809,0.120091,-0.129944,0.164500,...,0.418782,0.105937,-0.077353,0.145458,-0.104564,0.089727,0.196824,-0.061701,0.084830,0.228580
106,1.860231,0.957223,1.715041,-0.797369,-0.135379,1.626477,-0.979722,-0.422200,0.551456,1.199750,...,0.655804,-1.276193,-1.189938,0.286627,1.079101,0.935438,0.778107,-0.648928,-0.411956,2.801222


In [19]:
g = irf4Dc.columns.to_series().groupby(irf4Dc.dtypes).groups

In [25]:
irf4DcTrain=irf4Dc[irf4Dc.columns[5:]].loc[irf4IndesxList]

In [45]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances_argmin,silhouette_score
mbk=MiniBatchKMeans(init='k-means++', n_clusters=10,
                      n_init=10, max_no_improvement=10, verbose=0)

mbk.fit(irf4DcTrain)
labels = mbk.labels_

In [46]:
mbk_means_labels = pairwise_distances_argmin(irf4DcTrain, np.sort(mbk
                                                        .cluster_centers_, axis=0))

In [47]:
mbk_means_labels

array([4, 2, 6, ..., 2, 5, 2])

In [48]:
silhouette_score(irf4DcTrain, labels, metric='euclidean')

0.17972625248531049

# Testing for irf4

In [49]:
lstn_clusters=range(2,51)

In [52]:
for n in lstn_clusters:
    mbk=MiniBatchKMeans(init='k-means++', n_clusters=n,
                      n_init=10, max_no_improvement=10, verbose=0)

    mbk.fit(irf4DcTrain)
    labels = mbk.labels_
    print("n clusters: " + str(n))
    print("silhouette score: "str(silhouette_score(irf4DcTrain, labels, metric='euclidean')))
    

n clusters 2
0.315394078758
n clusters 3
0.314077762815
n clusters 4
0.2485019823
n clusters 5
0.194611061018
n clusters 6
0.193666072231
n clusters 7
0.210140001819
n clusters 8
0.154602929803
n clusters 9
0.190191010473
n clusters 10
0.176930433468
n clusters 11
0.108403163949
n clusters 12
0.138745336106
n clusters 13
0.0678031679945
n clusters 14
0.145713512117
n clusters 15
0.11925880629
n clusters 16
0.0853775026651
n clusters 17
0.111437243508
n clusters 18
0.0813902027617
n clusters 19
0.0973853958203
n clusters 20
0.0824602133179
n clusters 21
0.103153167575
n clusters 22
0.0813482550815
n clusters 23
0.0653624371515
n clusters 24
0.0915362057445
n clusters 25
0.0911195160189
n clusters 26
0.0682922917494
n clusters 27
0.10193143281
n clusters 28
0.100996732656
n clusters 29
0.0994679415846
n clusters 30
0.0769834767387
n clusters 31
0.0849709472643
n clusters 32
0.0948930448616
n clusters 33
0.0433289649433
n clusters 34
0.0805455197027
n clusters 35
0.0750815633104
n cluster

In [53]:
gc.collect()

159

In [83]:
from sklearn.cluster import AgglomerativeClustering
for n in lstn_clusters:
    ac=AgglomerativeClustering(n_clusters=n).fit(irf4DcTrain)
    labels = ac.labels_
    print("n clusters " + str(n))
    print("silhouette score: " +str(silhouette_score(irf4DcTrain, labels, metric='euclidean')))

n clusters 2
silhouette score: 0.345643248915
n clusters 3
silhouette score: 0.345980720105
n clusters 4
silhouette score: 0.173407760457
n clusters 5
silhouette score: 0.178583397321
n clusters 6
silhouette score: 0.113955327253
n clusters 7
silhouette score: 0.124487023648
n clusters 8
silhouette score: 0.130550304649
n clusters 9
silhouette score: 0.128944903998
n clusters 10
silhouette score: 0.132910094816
n clusters 11
silhouette score: 0.13459206439
n clusters 12
silhouette score: 0.113268770049
n clusters 13
silhouette score: 0.119402500664
n clusters 14
silhouette score: 0.113752394865
n clusters 15
silhouette score: 0.118613653057
n clusters 16
silhouette score: 0.117984526653
n clusters 17
silhouette score: 0.059101364108
n clusters 18
silhouette score: 0.0618059043887
n clusters 19
silhouette score: 0.0628743638572
n clusters 20
silhouette score: 0.0641001399339
n clusters 21
silhouette score: 0.065389446301
n clusters 22
silhouette score: 0.0668350608321
n clusters 23
silh